# Imports

## Dependencies

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from io import BytesIO
import datetime
import numpy as np
import configparser
import sys

## Micromet

In [2]:
#sys.path.append("G:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/MicroMet")
#sys.path.append("C:/Users/kladig/Documents/GitHub/MicroMet")
sys.path.append("../..")
import micromet

## Connection info and Config

In [ ]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

In [ ]:
manager = micromet.StationDataManager(config, engine)

In [ ]:


# Process data for all stations
site_folders = {#'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_Mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTL':'Pelican_Lake',
                }
manager.process_station_data(site_folders)

# Pull From Each Datalogger Using the CS Datalogger API

https://help.campbellsci.com/crbasic/cr1000x/Content/Info/webserverapicommands1.htm

In [ ]:
site_folders = {#'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_Mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTL':'Pelican_Lake',
                }

micromet.stat_dl_con_ul(site_folders, config, engine)

In [ ]:
site_folders = {'US-UTD':'Dugout_Ranch',
                'US-UTB':'BSF',
                'US-UTJ':'Bluff',
                'US-UTW':'Wellington',
                'US-UTE':'Escalante',
                'US-UTM':'Matheson',
                'US-UTP':'Phrag',
                'US-CdM':'Cedar_Mesa',
                'US-UTV':'Desert_View_Myton',
                'US-UTL':'Pelican_Lake',
                }

#stations = ['UTJ','UTV','UTP','UTW','UTM','CdM','UTL','UTE']
#compdf = {}

def stat_dl_con_ul(site_folders, engine):
    for stationid, name in site_folders.items():
        station = stationid.split('-')[-1]
        for dat in ['eddy','met']:
            if dat in config[station].keys():
                stationtime, comptime = micromet.get_times(station, config, loggertype=dat)
                am_df, pack_size = micromet.get_station_data(station, config, loggertype=dat)

                if am_df is not None:
                    am_df_filt = micromet.compare_sql_to_station(am_df, station, engine, loggertype=dat)
                    mindate = am_df_filt['TIMESTAMP_START'].min()
                    maxdate = am_df_filt['TIMESTAMP_START'].min()
                    raw_len = len(am_df)
                    am_df_len = len(am_df_filt)
                    am_df_filt = am_df_filt.rename(columns=str.lower)
                    am_df_filt.to_sql(f"amflux{dat}",con=engine, if_exists='append', index=False)
                    # Define variable names and values
                    variables = {'stationid' : stationid,
                                 'talbetype' : dat,
                                 'mindate' : mindate,
                                 'maxdate' : maxdate,
                                 'datasize_mb': pack_size,
                                 'stationdf_len': raw_len,
                                 'uploaddf_len': am_df_len,
                                 }

                    # Create a single-row dataframe
                    df = pd.DataFrame([variables])
                    df.to_sql(f"uploadstats", con=engine, if_exists='append', index=False)
                    # Display the dataframe
                else:
                    mindate = None
                    maxdate = None
                    raw_len = None
                    am_df_len = None

                print(dat)
                print(f"Station {station}")
                print(f"Mindate {mindate}  Maxdate {maxdate}")
                print(f"data size = {pack_size}")
                print(f"{am_df_len} vs {raw_len} rows")






In [ ]:
len(response_1.content)*1E-6

In [ ]:
mindate = am_df['TIMESTAMP_START'].min()
maxdate = am_df['TIMESTAMP_START'].min()

In [ ]:
#http://192.168.4.14/?command=ClockCheck
station = 'UTJ'
ip = config[station]['ip']
clk_url = f"http://{ip}:{port}/?"
#url = f"http://{ip}/tables.html?command=DataQuery&mode=since-record&format=toA5&uri=dl:Flux_AmeriFluxFormat&p1=0"
clk_args = {'command':'ClockCheck',
        'uri':'dl',
        'format':'json',
        }
clktimeresp = requests.get(clk_url, params=clk_args, auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw'])).json()
if 'time' in clktimeresp.keys():
    clktime = clktimeresp['time']
else:
    clktime = None

comptime = f"{datetime.datetime.now():%Y-%m-%d %H:%M:%S}"

In [ ]:
comptime

In [ ]:
station = 'UTE'
ip = config[station]['ip']
port = config[station]['met']
url = f"http://{ip}:{port}/tables.html?"
params = {'command':'DataQuery',
          'mode':'since-record',
          'format':'toA5',
          'uri':'dl:Statistics_AmeriFlux',
          'p1':0,
          }
response_1 = requests.get(url, params=params ,auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw']))
if response_1.status_code == 200:
    headers = pd.read_csv(BytesIO(response_1.content),skiprows=[0]).iloc[0:2,:].T
    raw_data = pd.read_csv(BytesIO(response_1.content),skiprows=[0,2,3])

